# 1. 뉴스기사 정보 스크랩하기(기사내용 포함x)
- Naver
- 원하는 언론사 선택 가능
- 날짜 선택 가능(시작과 끝)
- Key_word 이용해서 원하는 키워드로 검색 가능(해당 단어가 완전히 포함된 기사)

### 검색하기

In [33]:
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import math
import time

# Input값
key_word = '보장성강화'
datestart = '20170801'
dateend = '20181231'
news = ['KBS', 'MBC', 'SBS', 'JTBC', '연합뉴스', '조선일보', '중앙일보', '동아일보', '한국경제', '매일경제']

# chrome 드라이버 키고 naver 접속
driver = webdriver.Chrome('chromedriver.exe')
driver.get('http://naver.com')

driver.find_element_by_id('query').send_keys(key_word) # 검색어 입력
driver.find_element_by_id('search_btn').click() # 검색 클릭

driver.find_element_by_css_selector('div.lnb_menu').find_element_by_link_text('뉴스').click() # 뉴스 카테고리 클릭

# 검색옵션 클릭
driver.find_element_by_id('_search_option_btn').click()

# 기간 클릭
driver.find_element_by_link_text('기간').click()

# 기간 입력
driver.find_element_by_id('news_input_period_begin').click()
for i in datestart:
    driver.find_element_by_id('news_input_period_begin').send_keys(i)
    time.sleep(0.2)

driver.find_element_by_id('news_input_period_end').click()
for j in dateend:
    driver.find_element_by_id('news_input_period_end').send_keys(j)
    time.sleep(0.2)

# 적용 클릭
driver.find_element_by_css_selector('div.item_option.set_calendar').find_element_by_class_name('_btn_submit').click()

# 언론사 선택
driver.find_element_by_class_name('option_menu').find_element_by_link_text('언론사').click()

driver.find_element_by_id('order_cat')
for new in news:
    driver.find_element_by_xpath('//label[@title="{}"]'.format(new)).click()


driver.find_element_by_class_name('view_btn').find_element_by_css_selector('button.impact._submit_btn').click()

# 상세검색
driver.find_element_by_id('snb').find_element_by_link_text('상세검색').click()

driver.find_element_by_id('inpop1').send_keys(key_word)

driver.find_element_by_css_selector('button.btn_ft.ty_green._search').click()

# 오래된 순으로 정렬
driver.find_element_by_link_text('오래된순').click()

### 내용 가져오기 - 댓글 제외
- 제목
- URL
- 날짜
- 언론사 이름

In [186]:
# 내용 가져오기
title_list = []
url_list = []
date_list = []
reporter_list = []

total_number_raw = driver.find_element_by_css_selector('div.title_desc.all_my').text
total_number = int(re.search(r'/ (.*?$)', total_number_raw).group(1)[:-1].replace(',', ''))
total_page = math.ceil(total_number/10)

for c in range(total_page):
    time.sleep(0.3)
    print('\r{}/{}---title({})/url({})/reporter({})/date({})'.format(c+1, total_page,len(title_list), len(url_list), len(reporter_list), len(date_list)), end = '')
    new10 = driver.find_element_by_css_selector('ul.type01').find_elements_by_tag_name('li')
    
    for n in new10:
        title_list.append(n.find_element_by_css_selector('a._sp_each_title').get_attribute('title'))
        try:
            link = n.find_element_by_link_text('네이버뉴스').get_attribute('href')
            url_list.append(link)
            
            
        except:
            link = n.find_element_by_css_selector('a._sp_each_url._sp_each_title').get_attribute('href')
            url_list.append(link)
            
        
        for strings in n.find_element_by_css_selector('dd.txt_inline').text.split():
            
            for rep in news:
                if re.match(rep, strings):
                    reporter_list.append(rep)
                
            if re.search(r'^[0-9]+\.[0-9]+\.[0-9]+\.$', strings):
                date_list.append(strings[:-1].replace('.', '-'))
                
    if len(title_list) != len(reporter_list):
        print(c)
    driver.find_element_by_link_text('다음페이지').click()

### 내용 가져오기 - 댓글 포함

In [34]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 내용 가져오기
title_list = []
url_list = []
date_list = []
reporter_list = []
comment_list = []
while_error = 1

total_number_raw = driver.find_element_by_css_selector('div.title_desc.all_my').text
total_number = int(re.search(r'/ (.*?$)', total_number_raw).group(1)[:-1].replace(',', ''))
total_page = math.ceil(total_number/10)


current_window = driver.current_window_handle
for c in range(total_page):
    time.sleep(0.3)
    print('\r{}/{}---title({})/url({})/reporter({})/date({})/comment({}))'.format(c+1, total_page,len(title_list), len(url_list), len(reporter_list), len(date_list), len(comment_list)), end = '')
    new10 = driver.find_element_by_css_selector('ul.type01').find_elements_by_tag_name('li')
    
    for n in new10:
        
        title_list.append(n.find_element_by_css_selector('a._sp_each_title').get_attribute('title'))
        try:
            link = n.find_element_by_link_text('네이버뉴스').get_attribute('href')
            url_list.append(link)
            
            # 댓글 추가
            try:
                n.find_element_by_link_text('네이버뉴스').click()
                driver.switch_to.window(driver.window_handles[1])
                
                try:
                    element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "lo_txt"))
                    )
                except:
                    print('댓글창이 안 뜬다...')
                    
                while True:
                    if len(driver.find_element_by_css_selector('span.lo_txt').text) > 0:
                        while_error = 1
                        break
                    else:
                        print('\r아직 안 나왔어{}'.format(while_error), end = '')
                        while_error += 1
                        if while_error == 1000:
                            print('while 1000번 돌았어...')
                            while_error = 1
                            break                    
                    
                    try:
                        type(int(driver.find_element_by_css_selector('span.lo_txt').text)) == type(int(1))
                        break
                    except:
                        print('\r숫자가 안 나왔엉', end = '')
                        while_error += 1
                        if while_error == 1000:
                            print('while 1000번 돌았어...')
                            break
                            
                            
                            
                comment_list.append(driver.find_element_by_css_selector('span.lo_txt').text)
                driver.close()
                driver.switch_to.window(current_window)
            except:
                print('댓글실패')
                driver.close()
                driver.switch_to.window(current_window)
                comment_list.append(99999)
            
        except:
            link = n.find_element_by_css_selector('a._sp_each_url._sp_each_title').get_attribute('href')
            url_list.append(link)
            
        
        for strings in n.find_element_by_css_selector('dd.txt_inline').text.split():
            
            for rep in news:
                if re.match(rep, strings):
                    reporter_list.append(rep)
                
            if re.search(r'^[0-9]+\.[0-9]+\.[0-9]+\.$', strings):
                date_list.append(strings[:-1].replace('.', '-'))
                
    if len(title_list) != len(reporter_list):
        print(c)
    driver.find_element_by_link_text('다음페이지').click()

숫자가 안 나왔엉0itle(930)/url(930)/reporter(930)/date(930)/comment(930))댓글창이 안 뜬다...
댓글실패
숫자가 안 나왔엉-title(1160)/url(1160)/reporter(1160)/date(1160)/comment(1160))

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"다음페이지"}
  (Session info: chrome=72.0.3626.109)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)


### CSV파일로 저장하기

In [60]:
data = {'date' : date_list, 'title' : title_list, 'repoter' : reporter_list, 'URL' : url_list, 'comment' : comment_list}
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164 entries, 0 to 1163
Data columns (total 5 columns):
date       1164 non-null object
title      1164 non-null object
repoter    1164 non-null object
URL        1164 non-null object
comment    1164 non-null object
dtypes: object(5)
memory usage: 45.5+ KB


In [73]:
df.loc[df['comment'] == '댓글', 'comment'] = 0
df.loc[df['comment'] == 99999, 'comment'] = 0
file_name = '보장성강화_댓글추가_1164개.csv'
df.to_csv(file_name, encoding = 'utf-8', index = False)

### Comment와 content 파일 합치기

In [78]:
df_comment = pd.read_csv('보장성강화_댓글추가_1164개.csv', encoding = 'utf-8', engine = 'python')
df_comment.head()

,date,title,repoter,URL,comment
0,2017-08-05,첫 국산 수술로봇 '레보아이' 안전하고 믿을만할까,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,18
1,2017-08-09,"文 대통령 ""비급여 항목까지 건강보험 확대""",SBS,https://news.naver.com/main/read.nhn?mode=LSD&...,1
2,2017-08-09,'문재인 케어' 시동…미용·성형 빼고 모든 치료에 건보 적용,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,825
3,2017-08-09,[비급여 대수술] 1인당 연간 의료비 50만원에서 41만원으로 줄인다,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,45
4,2017-08-09,[비급여 대수술] 뇌혈관수술 2천만원→567만원,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,19


In [81]:
df_content = pd.read_csv('보장성강화_content포함_댓글제외.csv', encoding = 'utf-8', engine = 'python', names = ['date', 'title', 'reporter', 'URL', 'content'])
df_content.head()

,date,title,reporter,URL,content
0,2017-08-05,첫 국산 수술로봇 '레보아이' 안전하고 믿을만할까,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,개복수술보다 진료비 4~5배 비싼게 흠…건보 적용여부가 성공 열쇠 임상참여 나군호...
1,2017-08-09,"文 대통령 ""비급여 항목까지 건강보험 확대""",SBS,https://news.naver.com/main/read.nhn?mode=LSD&...,문재인 대통령은 오늘(9일) 서울의 한 대학병원을 찾아 치료비 부담을 국가가 나눠지...
2,2017-08-09,'문재인 케어' 시동…미용·성형 빼고 모든 치료에 건보 적용,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,"문 대통령, '울지 마세요'(서울=연합뉴스) 배재만 기자 = 문재인 대통령이 9일 ..."
3,2017-08-09,[비급여 대수술] 1인당 연간 의료비 50만원에서 41만원으로 줄인다,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,"문재인 대통령, 건강보험 정책 발표 (PG)[제작 최자윤] 일러스트 재난적 의료..."
4,2017-08-09,[비급여 대수술] 뇌혈관수술 2천만원→567만원,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,"문 대통령, '건강보험 보장은'(서울=연합뉴스) 배재만 기자 = 문재인 대통령이 9..."


In [107]:
result = pd.merge(df_comment, df_content, on = ['URL', 'date', 'title'], how = 'left')
result.drop('repoter', axis = 1, inplace = True)
result

,date,title,URL,comment,reporter,content
0,2017-08-05,첫 국산 수술로봇 '레보아이' 안전하고 믿을만할까,https://news.naver.com/main/read.nhn?mode=LSD&...,18,연합뉴스,개복수술보다 진료비 4~5배 비싼게 흠…건보 적용여부가 성공 열쇠 임상참여 나군호...
1,2017-08-09,"文 대통령 ""비급여 항목까지 건강보험 확대""",https://news.naver.com/main/read.nhn?mode=LSD&...,1,SBS,문재인 대통령은 오늘(9일) 서울의 한 대학병원을 찾아 치료비 부담을 국가가 나눠지...
2,2017-08-09,'문재인 케어' 시동…미용·성형 빼고 모든 치료에 건보 적용,https://news.naver.com/main/read.nhn?mode=LSD&...,825,연합뉴스,"문 대통령, '울지 마세요'(서울=연합뉴스) 배재만 기자 = 문재인 대통령이 9일 ..."
3,2017-08-09,[비급여 대수술] 1인당 연간 의료비 50만원에서 41만원으로 줄인다,https://news.naver.com/main/read.nhn?mode=LSD&...,45,연합뉴스,"문재인 대통령, 건강보험 정책 발표 (PG)[제작 최자윤] 일러스트 재난적 의료..."
4,2017-08-09,[비급여 대수술] 뇌혈관수술 2천만원→567만원,https://news.naver.com/main/read.nhn?mode=LSD&...,19,연합뉴스,"문 대통령, '건강보험 보장은'(서울=연합뉴스) 배재만 기자 = 문재인 대통령이 9..."
5,2017-08-09,[비급여 대수술] 건강보험 보장성 강화대책 Q&A,https://news.naver.com/main/read.nhn?mode=LSD&...,0,연합뉴스,"문재인 대통령, 건강보험 정책 발표 (PG)[제작 최자윤] 일러스트 (서울=연합..."
6,2017-08-09,"[비급여 대수술] 시민단체 ""공약후퇴"" vs 의료계 ""의료쇼핑 확산""",https://news.naver.com/main/read.nhn?mode=LSD&...,460,연합뉴스,"문재인 대통령, 건강보험 정책 발표 (PG)[제작 최자윤] 일러스트 (서울=연합..."
7,2017-08-09,文대통령 “2022년까지 건강보험 보장성 높여 의료비 걱정 없는 나라 만들 것”,https://news.naver.com/main/read.nhn?mode=LSD&...,4,KBS,"문재인 대통령은 9일(오늘) ""건강보험 보장성을 획기적으로 높여 가겠다""며 ""202..."
8,2017-08-09,"[전문] 文대통령 ""국민건강 지키는 나라다운 나라""…건보강화 발표문",https://news.naver.com/main/read.nhn?mode=LSD&...,22,연합뉴스,문재인 대통령 [연합뉴스 자료사진] (서울=연합뉴스) 박경준 기자 = 문재인 대...
9,2017-08-09,모든 치료비에 건보 적용…병원비 ‘가계 파탄’ 막는다,https://news.naver.com/main/read.nhn?mode=LSD&...,3,KBS,백혈병으로 투병하다 최근 조혈모세포이식술을 받은 53살 김모 씨. 수술 이후 75일...


In [109]:
result.to_excel('보장성강화_Naver뉴스_댓글+내용(1164개).xlsx', encoding = 'utf-8')

### 중간에 삭제된 뉴스 추출

In [111]:
df_1 = pd.read_csv('보장성강화_댓글제외.csv', engine = 'python', encoding = 'utf-8')
df_1.head()

,Unnamed: 0,date,title,repoter,URL
0,0,2017-08-05,첫 국산 수술로봇 '레보아이' 안전하고 믿을만할까,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...
1,1,2017-08-05,"文 대통령 ""비급여 항목까지 건강보험 확대""",연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2,2017-08-05,'문재인 케어' 시동…미용·성형 빼고 모든 치료에 건보 적용,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...
3,3,2017-08-05,[비급여 대수술] 1인당 연간 의료비 50만원에서 41만원으로 줄인다,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...
4,4,2017-08-05,[비급여 대수술] 뇌혈관수술 2천만원→567만원,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...


In [112]:
df_2 = pd.read_csv('보장성강화_댓글추가_1164개.csv', encoding = 'utf-8', engine = 'python')
df_2.head()

,date,title,repoter,URL,comment
0,2017-08-05,첫 국산 수술로봇 '레보아이' 안전하고 믿을만할까,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,18
1,2017-08-09,"文 대통령 ""비급여 항목까지 건강보험 확대""",SBS,https://news.naver.com/main/read.nhn?mode=LSD&...,1
2,2017-08-09,'문재인 케어' 시동…미용·성형 빼고 모든 치료에 건보 적용,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,825
3,2017-08-09,[비급여 대수술] 1인당 연간 의료비 50만원에서 41만원으로 줄인다,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,45
4,2017-08-09,[비급여 대수술] 뇌혈관수술 2천만원→567만원,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,19


In [120]:
a = df_1.URL.tolist()
b = df_2.URL.tolist()
c = list(set(a) - set(b))
c

['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=110&oid=025&aid=0002798778',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=025&aid=0002799502']

In [121]:
df_1[(df_1['URL'] == c[0]) | (df_1['URL'] == c[1])]

,Unnamed: 0,date,title,repoter,URL
673,673,2018-02-13,[송인한의 퍼스펙티브] 극단적 선택으로 떠밀리는 사회 … 사회적 타살이다,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...
676,676,2018-02-13,"건보공단, 18년 전 불법파업 해고자 등 6명 복직",연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...


# 2. 기사내용 스크랩
- 위에서 클롤링한 URL주소를 이용하여 해당 뉴스내용 스크랩
- 네이버뉴스로 통하는 기사만 가능
- 안되는 언론사 많음

### URL를 이용해 스크랩

In [226]:
content_list = []
content_url = []
comment_list = []
url_error_list = []

driver = webdriver.Chrome('chromedriver.exe')
for u in url_list:
    driver.get(u)
    try:
        content_list.append(driver.find_element_by_id('articleBodyContents').text.replace('\n', ' '))
        comment_list.append(driver.find_element_by_css_selector('span.lo_txt').text)
        content_url.append(u)
        print('\r진행상황 {}/{}'.format(len(content_list), len(url_list)), end = '')
    except:
        url_error_list.append(u)
        print('에러발생{}'.format(len(url_error_list)))

진행상황 939/1166에러발생1
진행상황 1165/1166

### 에러URL 추가적으로 크롤링

In [ ]:
driver = webdriver.Chrome('chromedriver.exe')
url_error_list2 = []
for u in url_error_list:
    driver.get(u)
    try:        
        content_list.append(driver.find_element_by_id('articleBodyContents').text.replace('\n', ' '))
        comment_list.append(driver.find_element_by_css_selector('span.lo_txt').text)
        content_url.append(u)
        print('\r진행상황 {}/{}'.format(len(content_list), len(url_list)), end = '')
    except:
        url_error_list2.append(u)
        print('또 에러발생{}'.format(len(url_error_list2)))

In [230]:
driver.get('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=001&aid=0009455538')

In [231]:
driver.find_element_by_css_selector('span.lo_txt').text

'18'

### CSV파일로 저장하기

In [246]:
df_content = pd.DataFrame({'URL' : content_url, 'content' : content_list})
df_content.head()

,URL,content
0,https://news.naver.com/main/read.nhn?mode=LSD&...,개복수술보다 진료비 4~5배 비싼게 흠…건보 적용여부가 성공 열쇠 임상참여 나군호...
1,https://news.naver.com/main/read.nhn?mode=LSD&...,문재인 대통령은 오늘(9일) 서울의 한 대학병원을 찾아 치료비 부담을 국가가 나눠지...
2,https://news.naver.com/main/read.nhn?mode=LSD&...,"문 대통령, '울지 마세요'(서울=연합뉴스) 배재만 기자 = 문재인 대통령이 9일 ..."
3,https://news.naver.com/main/read.nhn?mode=LSD&...,"문재인 대통령, 건강보험 정책 발표 (PG)[제작 최자윤] 일러스트 재난적 의료..."
4,https://news.naver.com/main/read.nhn?mode=LSD&...,"문 대통령, '건강보험 보장은'(서울=연합뉴스) 배재만 기자 = 문재인 대통령이 9..."


# 3. 동영상 링크 스크랩 - 스토리룸
- 해당 사이트의 동영상 링크와 해당 제목 스크랩

### 사이트 접속

In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import math
import time

URL_link = 'http://medicare1.nhis.or.kr/hongbo/static/html/minisite/sub/bj02.html'
driver = webdriver.Chrome('chromedriver.exe')

driver.get(URL_link)
driver.find_element_by_css_selector('ul.content-desc.story-desc.clearfix')

### 카테고리 세분화

In [152]:
c_list = []
for a in driver.find_elements_by_css_selector('ul.content-desc.story-desc.clearfix > li'):
    c_list.append(a.get_attribute('class').split()[1])
class_name_list = set(c_list)
class_name_list

{'dementia', 'etc', 'expense', 'mri', 'payment', 'support', 'system'}

### 자료수집

In [155]:
content_list = driver.find_elements_by_css_selector('ul.content-desc.story-desc.clearfix > li' )

title_list = []
url_list = []
class_list1 = []
class_list2 = []

for content in content_list:
    url_list.append(content.find_element_by_css_selector('div.story-title > a').get_attribute('href'))
    title_list.append(content.find_element_by_css_selector('div.story-title > a').text)
    class_list1.append(content.get_attribute('class').split()[0])
    class_list2.append(content.get_attribute('class').split()[1])
print(len(url_list), len(title_list), len(class_list1))

122 122 122


### CSV파일로 저장

In [160]:
data = {'title' : title_list, 'url' : url_list, 'class1' : class_list1, 'class2' : class_list2}
df = pd.DataFrame(data)
df.to_csv('스토리룸_유형+주제.csv', encoding = 'utf-8')

# 동영상 링크 스크램 - 뉴스룸
1. 카테고리별 형태가 달라서 각각 따로 처리

### 접속준비

In [202]:
driver = webdriver.Chrome('chromedriver.exe')

URL_link = 'http://medicare1.nhis.or.kr/hongbo/static/html/minisite/sub/bj03_{}.html'
title_list = []
url_list = []
class_list = []
cate = ['브리핑', '보도자료', '보도해명/설명자료', 'TV,라디오 등 방송인터뷰' ,'팩트체크']

### 자료수집 - 브리핑

In [204]:
driver.get(URL_link.format(1))
content_list = driver.find_elements_by_css_selector('ul.content-desc > li')

for content in content_list:
    title_list.append(content.find_element_by_css_selector('div.story-title > a').text)
    url_list.append(content.find_element_by_css_selector('div.story-title > a').get_attribute('href'))
    class_list.append(cate[0])
len(url_list)

1

### 자료수집 - 보도자료

In [205]:
driver.get(URL_link.format(2))
content_list = driver.find_elements_by_css_selector('ul.content-desc > li.type-news.news-tab2 > ol > li > a')

for content in content_list:
    title_list.append(content.text[2:])
    url_list.append(content.get_attribute('href'))
    class_list.append(cate[1])
print(len(title_list), len(url_list), len(class_list))

45 45 45


### 자료수집 - 보도해명/설명자료

In [206]:
driver.get(URL_link.format(3))
content_list = driver.find_elements_by_css_selector('ul.content-desc > li.type-news.news-tab2 > ol > li > a')

for content in content_list:
    title_list.append(content.text[2:])
    url_list.append(content.get_attribute('href'))
    class_list.append(cate[2])
print(len(title_list), len(url_list), len(class_list))

74 74 74


### 자료수집 - TV,라디오 등 방송인터뷰

In [207]:
driver.get(URL_link.format(4))
content_list = driver.find_elements_by_css_selector('ul.content-desc > li')

for content in content_list:
    title_list.append(content.find_element_by_css_selector('div.story-title > a').text)
    url_list.append(content.find_element_by_css_selector('div.story-title > a').get_attribute('href'))
    class_list.append(cate[3])
len(url_list)

125

### 자료수집 - 팩트체크

In [208]:
driver.get(URL_link.format(5))
content_list = driver.find_elements_by_css_selector('ul.content-desc.story-desc.list3.clearfix > li')

for content in content_list:
    title_list.append(content.find_element_by_css_selector('div.story-title > a').text)
    url_list.append(content.find_element_by_css_selector('div.story-title > a').get_attribute('href'))
    class_list.append(cate[4])
len(url_list)

127

### CSV파일로 저장

In [210]:
data = {'title' : title_list, 'url' : url_list, 'class' : class_list}
df = pd.DataFrame(data)
df.to_csv('뉴스룸.csv', encoding = 'utf-8')